### **Connect to the workspace**

In [3]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="",
    resource_group_name="",
    workspace_name="",
)

### **Compute Resource**

In [13]:
from azure.ai.ml.entities import AmlCompute

gpu_compute_target = "safira-gpu"

try:
    # let's see if the compute target already exists
    gpu_cluster = ml_client.compute.get(gpu_compute_target)
    print(
        f"You already have a cluster named {gpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new gpu compute target...")

    gpu_cluster = AmlCompute(
        name="gpu-cluster",
        type="amlcompute",
        size="STANDARD_NC6",  # 1 x NVIDIA Tesla K80
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )

    gpu_cluster = ml_client.begin_create_or_update(gpu_cluster)



You already have a cluster named safira-gpu, we'll reuse it as is.


In [14]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

### **Enviorment**

In [15]:
from azure.ai.ml.entities import Environment

custom_env_name = "ml-use-case-1"

pipeline_job_env = Environment(
    name=custom_env_name,
    version="12",
)


print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name ml-use-case-1 is registered to workspace, the environment version is 12


In [16]:
import os

data_prep_src_dir = "./utils"
os.makedirs(data_prep_src_dir, exist_ok=True)

### **Data Preprocess Job**

In [17]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component = command(
    name="data_prep_use_case1",
    display_name="Data preparation for training",
    description="reads a input, give the BIO tag and word embedding for training",
    inputs={
        "data": Input(type="uri_file"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=data_prep_src_dir,
    command="""python data_process.py \
            --data ${{inputs.data}} \
            --train_data ${{outputs.train_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute= gpu_compute_target,
)

In [18]:
# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component.component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Component data_prep_use_case1 with Version 2023-05-30-09-26-51-5337825 is registered


### **Train model Job**

In [19]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

train_component = command(
    name="train_use_case1",
    display_name="Training",
    description="gives an output",
    inputs={
        "train_data": Input(type="uri_folder"),
    },
    outputs= dict(
        model=Output(type="uri_folder", mode="rw_mount")
    ),
    code='./',
    # The source folder of the component
    command="""python train.py \
            --train_data ${{inputs.train_data}} \
            --model ${{outputs.model}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute= gpu_compute_target,
)

In [20]:
# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component.component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)


Uploading Use Case 1 (0.16 MBs): 100%|██████████| 163181/163181 [00:00<00:00, 1107747.61it/s]




Component train_use_case1 with Version 2023-05-30-09-26-55-2800773 is registered


### **Pipeline**

In [21]:
# the dsl decorator tells the sdk that we are defining an Azure ML pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute=gpu_compute_target,
    description="Use case 1 pipeline",
)
def usecase1_pipeline(
    pipeline_job_data_input,
):
    # using data_prep_function like a python call with its own inputs
    data_prep_job = data_prep_component(
        data=pipeline_job_data_input,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
    }

In [22]:
# Let's instantiate the pipeline with the parameters of our choice
pipeline = usecase1_pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=''),
)

In [23]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="uc1_registered_components",
)

### **Deployment**

In [32]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "uc1-" + str(uuid.uuid4())[:8]

In [33]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint for uc1",
    auth_mode="key",
    tags={
        "training_dataset": "product dataste",
        "model_type": "OpenTag",
    },
)

endpoint_result = ml_client.begin_create_or_update(endpoint).result()

print(
    f"Endpint {endpoint_result.name} provisioning state: {endpoint_result.provisioning_state}"
)

Endpint uc1-54cfa399 provisioning state: Succeeded


In [34]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "uc1-54cfa399" with provisioning state "Succeeded" is retrieved


In [35]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name="use-case1-model")]
)

In [36]:
env = Environment(
    conda_file=os.path.join("./dependencies", "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

In [38]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential

# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name="use-case1-model", version=latest_model_version)


# create an online deployment.
red_deployment_uc1 = ManagedOnlineDeployment(
    name="uc1",
    endpoint_name=online_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="./dependencies",
        scoring_script="score.py"
    ),
    environment=env,
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

red_deployment_results = ml_client.online_deployments.begin_create_or_update(
    red_deployment_uc1
).result()

print(
    f"Deployment {red_deployment_results.name} provisioning state: {red_deployment_results.provisioning_state}"
)

Check: endpoint uc1-54cfa399 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..........................................................................................................Deployment uc1 provisioning state: Succeeded


### **Inference**

In [1]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_result.name,
    deployment_name=red_deployment_uc1.name,
    request_file=os.path.join("./dependencies", "sample.json"),
)

NameError: name 'ml_client' is not defined